# Classification problem

## Instructions

-  We consider the dataset file <code>**dataset.csv**</code>, which is contained in the <code>**loan-prediction**</code> directory

-  A description of the dataset is available in the <code>**README.txt**</code> file on the same directory.

-  **GOAL:** Use information from past loan applicants contained in <code>**dataset.csv**</code> to predict whether a _new_ applicant should be granted a loan or not.

## Dataset preparation

In [1]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Import stats module from scipy, which contains a large number of probability distributions as well as an exhaustive library of statistical functions.
import scipy.stats as stats

# need to ignore the warnings
import warnings

### Data collection

In [2]:
# Path to the local dataset file (YOURS MAY BE DIFFERENT!)
DATASET_PATH = './data/loan-prediction/dataset.csv'

# Load the dataset with Pandas
data = pd.read_csv(DATASET_PATH, sep=',', index_col='Loan_ID')

# show result
data.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
Loan_ID,,,,,,,,,,,,
LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


### Handling missing values

The first thing we might do is to replace the NA values with the mean of all the values (in the case of numerical values). The reality is that with the presence of _outliers_, the mean might not be the best choice. The __median__ is a better solution, being indeed robust to the outliers in the dataset.

In [3]:
from pandas.api.types import is_numeric_dtype

# removed NA values
data = data.apply(lambda x:
                  x.fillna(x.median()) if is_numeric_dtype(x)
                  else x.fillna(x.mode().iloc[0]))

# show result
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,145.752443,342.410423,0.855049
std,6109.041673,2926.248369,84.107233,64.428629,0.352339
min,150.000000,0.000000,9.000000,12.000000,0.000000
25%,2877.500000,0.000000,100.250000,360.000000,1.000000
50%,3812.500000,1188.500000,128.000000,360.000000,1.000000
75%,5795.000000,2297.250000,164.750000,360.000000,1.000000
max,81000.000000,41667.000000,700.000000,480.000000,1.000000


### Encoding categorical features - _One-hot Encoding_

Categorical values should be transformed into numerical values to be used in the machine-learning pipeline. Not all the ML models can support categorical values.

This procedure is achieved by the <tt>get_dummies</tt> function.


In [5]:
# get categorical features
# not calculating Loan_Status beacuse it is binary but it is not numerical
categorical_features = [col for col in data.columns if not is_numeric_dtype(data[col]) and col != 'Loan_Status']

# get dummy function
data_with_dummy = pd.get_dummies(data, columns=categorical_features)

# check result
data_with_dummy.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True
LP001003,4583,1508.0,128.0,360.0,1.0,N,False,True,False,True,...,True,False,False,True,False,True,False,True,False,False
LP001005,3000,0.0,66.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,True,False,False,True,False,False,True
LP001006,2583,2358.0,120.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,False,True,True,False,False,False,True
LP001008,6000,0.0,141.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True


Move the predicted column to the last

In [12]:
# move predicted column to last
columns = data_with_dummy.columns.tolist()
columns.insert(len(columns), columns.pop(columns.index("Loan_Status")))
data_with_dummy = data_with_dummy.loc[:, columns]

# check result
data_with_dummy.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,Y
LP001003,4583,1508.0,128.0,360.0,1.0,False,True,False,True,False,...,False,False,True,False,True,False,True,False,False,N
LP001005,3000,0.0,66.0,360.0,1.0,False,True,False,True,True,...,False,False,True,False,False,True,False,False,True,Y
LP001006,2583,2358.0,120.0,360.0,1.0,False,True,False,True,True,...,False,False,False,True,True,False,False,False,True,Y
LP001008,6000,0.0,141.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,Y


### Encoding binary class label

To make the binary class labels in a numerical value, first identify the col and the two possible values. Then replace the with 1 and -1.

In [21]:
# replace data with dummies
data = data_with_dummy

# replace binary labels with binary numerical values
data.Loan_Status = data.Loan_Status.map(lambda x: 1 if x == 'Y' else -1)

# check result
data.head()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Female,Gender_Male,Married_No,Married_Yes,Dependents_0,...,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,Loan_Status
Loan_ID,,,,,,,,,,,,,,,,,,,,,
LP001002,5849,0.0,128.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,1
LP001003,4583,1508.0,128.0,360.0,1.0,False,True,False,True,False,...,False,False,True,False,True,False,True,False,False,-1
LP001005,3000,0.0,66.0,360.0,1.0,False,True,False,True,True,...,False,False,True,False,False,True,False,False,True,1
LP001006,2583,2358.0,120.0,360.0,1.0,False,True,False,True,True,...,False,False,False,True,True,False,False,False,True,1
LP001008,6000,0.0,141.0,360.0,1.0,False,True,True,False,True,...,False,False,True,False,True,False,False,False,True,1


## Build the model

In [29]:
from sklearn.metrics            import get_scorer
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn                    import tree

# cross validation
from sklearn.model_selection    import KFold
from sklearn.model_selection    import StratifiedKFold
from sklearn.model_selection    import cross_val_score
from sklearn.model_selection    import cross_validate
from sklearn.model_selection    import train_test_split

# hyperparams optimization
from sklearn.model_selection    import GridSearchCV
from sklearn.metrics            import accuracy_score
from sklearn.metrics            import roc_auc_score
from sklearn.metrics            import f1_score
from sklearn.metrics            import precision_score
from sklearn.metrics            import classification_report
from sklearn.metrics            import explained_variance_score

# models
from sklearn.linear_model       import LogisticRegression
from sklearn.svm                import LinearSVC
from sklearn.svm                import SVC
from sklearn.tree               import DecisionTreeClassifier
from sklearn.tree               import DecisionTreeRegressor
from sklearn.neighbors          import KNeighborsRegressor
from sklearn.neighbors          import KNeighborsClassifier
from sklearn.ensemble           import RandomForestClassifier
from sklearn.ensemble           import AdaBoostClassifier
from sklearn.ensemble           import GradientBoostingClassifier
from sklearn.ensemble           import RandomForestRegressor

#from sklearn.externals import joblib

### Split the dataset

In [24]:
# extract dataset X from the DataFrame
X = data.iloc[:, : -1]
X.head()

# extract the target
y = data.iloc[:, -1]
y.head()

Loan_ID
LP001002    1
LP001003   -1
LP001005    1
LP001006    1
LP001008    1
Name: Loan_Status, dtype: int64

Let's split our dataset with __scikit-learn__ <tt>train_test_split</tt> function, which splits the input dataset into a training set and a test set, respectively.

We want the training set to account for 80% of the original dataset, whilst 
the test set to account for the remaining 20%.

Additionally, we would like to take advantage of _stratified_ sampling to obtain the same target distribution in both the training and the test sets.


In [25]:
# fixed random state
RND_SEED = 159

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=RND_SEED)

### Evaluate function

We can create a function such that it will print the evaluation of the prediction.

In [30]:
"""
General function used to assess the quality of predictions in terms of two scores:
 - accuracy 
 - ROC AUC (Area Under the ROC Curve)
"""
def evaluate(true_values, predicted_values):
        
        print("Precision")
        print(f"   {precision_score(true_values, predicted_values) }")
        
        print("F1-score")
        print(f"   {f1_score(true_values, predicted_values) }")

### Cross-validation

In [32]:
# ignore warnings
warnings.filterwarnings('ignore')

# create the model
model = KNeighborsClassifier()

# perform cross validation
cross_validation = cross_validate(model, X, y,
                                  cv = 10,
                                  scoring = ('precision', 'f1'),
                                  return_train_score=True)

# print result
pd.DataFrame(cross_validation)

,fit_time,score_time,test_precision,train_precision,test_f1,train_f1
0,0.010970,1.474781,0.709091,0.757709,0.795918,0.825930
1,0.001995,0.012966,0.703704,0.759825,0.783505,0.831541
2,0.003990,0.014958,0.644444,0.750000,0.666667,0.818182
3,0.003987,0.021945,0.660000,0.757174,0.717391,0.823529
4,0.003989,0.015958,0.673077,0.746269,0.744681,0.824499
5,0.005982,0.020946,0.685185,0.754923,0.770833,0.824373
6,0.002995,0.015952,0.673077,0.760000,0.744681,0.824096
7,0.002993,0.014986,0.666667,0.749465,0.750000,0.826446
8,0.003986,0.009973,0.714286,0.742004,0.769231,0.819788
9,0.002990,0.010972,0.688889,0.756044,0.712644,0.823952


In [35]:
# print averages
print("Mean of the TEST SET scores")
print(f"Precision: {np.mean(cross_validation['test_precision']): .3f}")
print(f"F1-score : {np.mean(cross_validation['test_f1']): .3f}")

Mean of the TEST SET scores
Precision:  0.682
F1-score :  0.746


### K-fold cross-validation

The k-fold cross-validation is an improved validation test where the dataset is divided into $K$ parts and at every iteration a part is used as a test set and the others $K - 1$ as a train set.

In [36]:
# define the model
model = KNeighborsClassifier()

# define the k-fold validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# perform cross validation
cross_validation = cross_validate(model, X, y,
                                  cv = k_fold,
                                  scoring=('precision', 'f1'),
                                  return_train_score=True)

# print result
pd.DataFrame(cross_validation)

,fit_time,score_time,test_precision,train_precision,test_f1,train_f1
0,0.006981,0.046874,0.581818,0.757895,0.719101,0.834299
1,0.006982,0.021941,0.620000,0.760965,0.666667,0.831138
2,0.008976,0.018949,0.700000,0.750538,0.777778,0.824085
3,0.002992,0.012966,0.692308,0.748359,0.750000,0.819162
4,0.006984,0.018947,0.645833,0.758621,0.704545,0.832151
5,0.005984,0.009971,0.666667,0.756522,0.747253,0.826603
6,0.004987,0.012966,0.787234,0.739224,0.795699,0.816667
7,0.004986,0.013962,0.711111,0.760440,0.735632,0.828743
8,0.002987,0.018953,0.796296,0.740576,0.843137,0.809697
9,0.005004,0.018924,0.730769,0.754386,0.783505,0.825930


In [37]:
# print averages
print("Mean of the TEST SET scores")
print(f"Precision: {np.mean(cross_validation['test_precision']): .3f}")
print(f"F1-score : {np.mean(cross_validation['test_f1']): .3f}")

Mean of the TEST SET scores
Precision:  0.693
F1-score :  0.752


### Stratified k-fold cross-validation

An even better option is to use a stratified k-fold validation. This variant splits the dataset in a way such that every fold contains the same proportion of features.

In [39]:
# define the model
model = KNeighborsClassifier()

# define stratified k-fold
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# perform the cross-validation
cross_validation = cross_validate(model, X, y,
                                  cv = k_fold,
                                  scoring = ('precision', 'f1'),
                                  return_train_score=True)

# print result
pd.DataFrame(cross_validation)

,fit_time,score_time,test_precision,train_precision,test_f1,train_f1
0,0.006981,0.036901,0.698113,0.751092,0.770833,0.821983
1,0.003988,0.021942,0.693878,0.759382,0.739130,0.826923
2,0.007981,0.017950,0.677966,0.759912,0.792079,0.827338
3,0.004983,0.017953,0.705882,0.754881,0.774194,0.827586
4,0.006982,0.011968,0.703704,0.751092,0.791667,0.821002
5,0.002991,0.017952,0.760000,0.746269,0.826087,0.824499
6,0.002992,0.014959,0.653061,0.762009,0.703297,0.832936
7,0.007979,0.030917,0.692308,0.752711,0.765957,0.825208
8,0.004986,0.023936,0.690909,0.743590,0.783505,0.820755
9,0.004990,0.022937,0.673913,0.752174,0.704545,0.823810


In [40]:
# print averages
print("Mean of the TEST SET scores")
print(f"Precision: {np.mean(cross_validation['test_precision']): .3f}")
print(f"F1-score : {np.mean(cross_validation['test_f1']): .3f}")

Mean of the TEST SET scores
Precision:  0.695
F1-score :  0.765


## Comparing different models

There might be a situation where different models can be compared to see which one fits better to the classification problem we need to solve.

### Select the best hyper-params of a fixed family of model

In this first case, we study the influence different hyper-params have on the same family model (logistic regression) and choose the best

In [48]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=RND_SEED)

# dictonary of models and hyperparam
models_and_hyperparams = {
    'KNeighborsClassifier' : (KNeighborsClassifier(), {
        'n_neighbors' : [1, 5, 10, 25, 50, 100]
    })
}

# define folds
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# get the model
model = models_and_hyperparams['KNeighborsClassifier'][0]

# get dictionary of hyperparameters
hyperparams = models_and_hyperparams['KNeighborsClassifier'][1]

# use Grid Search to compare all the combination
grid_search = GridSearchCV(model, hyperparams,
                           cv=k_fold,
                           scoring=('f1'),
                           verbose=True,
                           return_train_score=True)

# find the solution
grid_search.fit(X_train, y_train)

# display result
pd.DataFrame(grid_search.cv_results_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
0,0.005286,0.002095,0.017752,0.006367,1,{'n_neighbors': 1},0.771429,0.750000,0.800000,0.789474,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
1,0.004687,0.002233,0.012167,0.002918,5,{'n_neighbors': 5},0.835443,0.815789,0.814815,0.746667,...,0.839363,0.844702,0.829912,0.842415,0.841499,0.839827,0.838150,0.838897,0.838264,0.004264
2,0.003491,0.001115,0.008378,0.002757,10,{'n_neighbors': 10},0.779221,0.794872,0.829268,0.789474,...,0.825036,0.830680,0.826979,0.838897,0.825899,0.837143,0.824891,0.829268,0.828863,0.006231
3,0.003292,0.001342,0.009774,0.001775,25,{'n_neighbors': 25},0.823529,0.833333,0.833333,0.833333,...,0.827128,0.826029,0.824632,0.826029,0.828685,0.822903,0.825566,0.829787,0.826079,0.001960
4,0.003591,0.001277,0.012970,0.002952,50,{'n_neighbors': 50},0.823529,0.833333,0.833333,0.833333,...,0.826029,0.826029,0.826029,0.826029,0.827586,0.827586,0.827586,0.827586,0.826762,0.000744
5,0.003890,0.001442,0.009270,0.001414,100,{'n_neighbors': 100},0.823529,0.833333,0.833333,0.833333,...,0.826029,0.826029,0.826029,0.826029,0.827586,0.827586,0.827586,0.827586,0.826762,0.000744


In [47]:
# get best combination
print(f"Best hyperparameter:")
print(grid_search.best_params_)
print(f"Best F1-score: {grid_search.best_score_:.3}")

Best hyperparameter:
{'n_neighbors': 25}
Best F1-score: 0.824


In [49]:
# define model with best hyperparams
model = KNeighborsClassifier(n_neighbors=grid_search.best_params_['n_neighbors'])

# train model on whole dataset
model.fit(X_train, y_train)

# evaluate the prediction capabilities
evaluate(y_test, model.predict(X_test))

Precision
   0.6178861788617886
F1-score
   0.7638190954773869


### Best model from fixed hyper-params

Here we fix the hyper-params for each model (we use the default params) and compare the different models

In [52]:
# ignore warnings
warnings.filterwarnings('ignore')

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=RND_SEED)

# define models
models = {
    'LogisticRegression'          : LogisticRegression(),
    'KNeighborsClassifier'      : KNeighborsClassifier(),
    'DecisionTreeClassifier'    : DecisionTreeClassifier()
}

# define folds
k_fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=RND_SEED)

# cross validate the models manually
cross_validation_scores = dict()
for model_name, model in models.items():
    cross_validation_scores[model_name] = cross_val_score(model, X_train, y_train,
                                                          cv=k_fold,
                                                          scoring=('precision'))

# save results in a DataFrame
cross_validation_scores = pd.DataFrame(cross_validation_scores).transpose()

# compute mean and std-dev
cross_validation_scores['mean'] = np.mean(cross_validation_scores, axis=1)
cross_validation_scores['std-dev'] = np.std(cross_validation_scores, axis=1)
cross_validation_scores = cross_validation_scores.sort_values(['mean', 'std-dev'], ascending=False)

# print result
cross_validation_scores

,0,1,2,3,4,5,6,7,8,9,mean,std-dev
LogisticRegression,0.853659,0.772727,0.829268,0.853659,0.825000,0.772727,0.825000,0.772727,0.772727,0.868421,0.814592,0.034893
DecisionTreeClassifier,0.793103,0.777778,0.833333,0.842105,0.851852,0.742857,0.800000,0.756757,0.764706,0.833333,0.799582,0.035168
KNeighborsClassifier,0.750000,0.756098,0.717391,0.700000,0.780488,0.697674,0.692308,0.697674,0.697674,0.666667,0.715597,0.031882


By comparing the mean and the standard deviation we can deduce that the best classifier is the logistic regression. We now need to train the model on the whole train set (so far we trained in the cross-validation folds only). After training in the whole train set, we predict the values on the test set and evaluate the result. There is nothing more we can do.

In [56]:
# save the best model
model = models[cross_validation_scores.index[0]]

# re-train the best model on the whole train set
model.fit(X_train, y_train)

# evaluate the test set predicion
evaluate(y_test, model.predict(X_test))

Precision
   0.7281553398058253
F1-score
   0.8379888268156425
